### Brain Extraction

Once the data is downloaded, this script is used extract the brain from the skull. We used Deep_MRI_brain_extraction [Deep MRI Brain Extraction](https://github.com/GUR9000/Deep_MRI_brain_extraction)
(we used the commit version 7c2db1e).

You **must** have a separate environment with the packages listed below because it an older model and uses Theano and older version of python: 

- **Python 2.7**
- theano 0.8.2
- nose
- h5py 2.9.0
- nibabel 2.5.2

**PYTHON 2.7 AND THESE PACKAGES IS ONLY FOR THIS NOTEBOOK** \
**ANY OTHER SCRIPT OUTSIDE THIS NOTEBOOK FOLLOWS THE Readme.md**

You should also have `~/.theanorc` configured. For example, this demo uses CPU version of the brain extraction model and the following snippet of code is within `~/.theanorc`:

    [global]
    floatX=float32 
    device=cpu

We highly recommend configuring theano so that it uses GPU by following the instruction within their repository [repository](https://github.com/GUR9000/Deep_MRI_brain_extraction). However, feel free to any other brain extraction algorithm.

The scripts within this notebook creates 3 folder `T1_mask`, `T2_mask` and `PD_mask` within `../data/processed_IXI/`. Each mask has the following naming convention `IXI###-HOSPITAL-####_mask.nii.gz`. For example, subject `../data/processed_IXI/T1/IXI261-HH-1704.nii` will have a corresponding mask  `../data/processed_IXI/T1_mask/IXI261-HH-1704_mask.nii.gz`.

### Skullstripping

Clones the repository and predict the mask

In [ ]:
import shutil
shutil.rmtree('./misc/', ignore_errors=True)

"""Cloning repository"""
!git clone https://github.com/GUR9000/Deep_MRI_brain_extraction.git ./misc/Deep_MRI_brain_extraction/
!cd ./misc/Deep_MRI_brain_extraction/ && git checkout 7c2db1e

"""Remove matplotlib"""
_file = "./misc/Deep_MRI_brain_extraction/NNet_Core/NN_ConvNet.py"
a_file = open(_file, "r")
list_of_lines = a_file.readlines()
list_of_lines[87] = '#from matplotlib import pyplot as plot\n'
a_file = open(_file, "w")
a_file.writelines(list_of_lines)
a_file.close()

from pathlib import Path
Path('../data/processed_IXI/T1_mask/').mkdir(parents=True, exist_ok=True)
Path('../data/processed_IXI/T2_mask/').mkdir(parents=True, exist_ok=True)
Path('../data/processed_IXI/PD_mask/').mkdir(parents=True, exist_ok=True)

!python ./misc/Deep_MRI_brain_extraction/deep3Dpredict.py \
        -n ./misc/Deep_MRI_brain_extraction/OASIS_ISBR_LPBA40__trained_CNN.save \
        -i ../data/processed_IXI/T1/ \
        -o ../data/processed_IXI/T1_mask/ \
        -gridsize 16

!python ./misc/Deep_MRI_brain_extraction/deep3Dpredict.py \
        -n ./misc/Deep_MRI_brain_extraction/OASIS_ISBR_LPBA40__trained_CNN.save \
        -i ../data/processed_IXI/T2/ \
        -o ../data/processed_IXI/T2_mask/ \
        -gridsize 16

!python ./misc/Deep_MRI_brain_extraction/deep3Dpredict.py \
        -n ./misc/Deep_MRI_brain_extraction/OASIS_ISBR_LPBA40__trained_CNN.save \
        -i ../data/processed_IXI/PD/ \
        -o ../data/processed_IXI/PD_mask/ \
        -gridsize 16


### Renaming files

Rename each file and delete unnecessary outputs

In [ ]:
for m in ['T1_mask/', 'T2_mask/', 'PD_mask/']:
    _path = '../data/processed_IXI/'+m
    for d in os.listdir(_path):
        if 'mask' in d:
            os.rename(_path+d, 
                      _path+d.split('_')[1]+'_mask.nii.gz')
        else: 
            os.remove(_path+d)
            print('a')